In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns
from datetime import datetime
import math, statistics

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/df_preprocessed.csv", index_col = 0)

/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


# Risk Ratio

LABELING 
https://boxnwhis.kr/2014/11/11/measure_churn_for_individual_users.html
https://boxnwhis.kr/2015/02/09/predict_churns.html

"""
사용자 A와 B는 똑같이 7일 동안 접속하지 않았다. 만약 서비스의 일괄적인 이탈 기준이 5일이라면 두 명의 사용자는 똑같은 이탈 유저가 된다.
그러나, 서로 다른 접속 주기를 고려한다면 A와 B는 다른 이탈 가능성을 갖는다. 단순히 마지막 접속 경과일이 특정 숫자를 넘으면 이탈 유저이고 
그렇지 않으면 이탈 유저가 아니라는 식의 구분은 주어진 정보를 충분히 사용하지 않는 것이다.

이러한 문제를 해결하기 위해서 개별 사용자의 접속 주기와 최종 접속 경과일을 함께 고려한 척도가 필요하다. 
하나의 대안으로 [최종 접속 경과일] / [평균 접속 주기]라는 새로운 지표를 만들고 위험 비율(risk ratio)이라고 부른다. 
사용자 A와 B에 대해서 새로운 지표를 계산하면 아래 그림과 같다.

A의 경우, 평균적으로 2일마다 서비스에 접속했으나 최근 7일 동안 접속하지 않았으므로 7 / 2 = 3.5의 위험 비율을 갖는다. 
평균 접속 주기의 3.5배 시간만큼 접속하지 않았다는 의미가 된다. B의 경우, 평균적으로 4일마다 서비스에 접속했으나
최근 7일 동안 접속하지 않았으므로 7 / 4 = 1.8의 위험 비율을 갖는다. 평균보다 1.8배의 시간동안 접속이 없었다.
이 두 가지 숫자를 비교하면 사용자 B보다는 A가 이탈 가능성이 높다.
"""
위험비율 = [최종 접속 경과일] / [평균 접속 주기]
==> 평균접속주기 == NaN인 유저(첫접속만 하고 이탈한 유저)의 값은 엄청 작은 값으로 임의로 설정하여 위험비율을 극단적으로 높인다.
==> 혹은 아예 데이터에서 삭제하는 방법도 있음

In [ ]:
# NaN값 epsilon 치환
df = df.fillna(1*(10**(-4)))
df.isnull().sum()

gamer_id       0
game_id        0
inDate         0
category       0
time           0
action         0
day            0
login_total    0
diff           0
login_mean     0
dtype: int64

### 최종접속경과일 (가장 최근 diff)

In [ ]:
# 유저별 날짜별 diff 데이터프레임 생성
df_diff = df[['gamer_id', 'inDate', 'diff']]
df_diff = df_diff.groupby(by=['gamer_id', 'inDate']).mean().reset_index()

In [ ]:
df_diff

gamer_id      inDate  diff
0        000010f658e9f47d01fd060a769e1d95395ac7c97c357d...  2022-01-05     0
1        000010f658e9f47d01fd060a769e1d95395ac7c97c357d...  2022-01-06     1
2        000010f658e9f47d01fd060a769e1d95395ac7c97c357d...  2022-01-07     1
3        000010f658e9f47d01fd060a769e1d95395ac7c97c357d...  2022-01-08     1
4        000012d4f585a9bb5d6975e157b79b3538b04885ac19c6...  2022-01-01     0
...                                                    ...         ...   ...
1419953  ffff55ab88728761706a92ef75d68394c5dad8edd66818...  2022-01-01     0
1419954  ffff84a159d96864ec6ee5ae6c9c4386d3060666bcc5a1...  2021-12-30     0
1419955  ffffebdf9cff7c9d118dbb90fe145fbeb61c4978e1692d...  2022-01-01     0
1419956  fffff1f88815e0ba807c3ff28ebcb94d9dbd8fce805eae...  2022-01-10     0
1419957  fffff98c3ca2713d099b6fa011b43b4b5834e28234c2c6...  2021-12-29     0

[1419958 rows x 3 columns]

In [ ]:
# merge를 위한 데이터프레임의 데이터인 유저별 최종접속경과일 딕셔너리 생성
merge_dict = {}
for i in range(len(df_diff['gamer_id'])):
    if i == 0:
        pass
    elif df_diff['gamer_id'][i] == df_diff['gamer_id'][i-1]:
        pass
    else: # 이전 gamer_id와 다르면
        if int(df_diff['inDate'].iloc[i-1][5]+df_diff['inDate'].iloc[i-1][6]) == 12: # 12월일 경우
            plus = 31 - int(df_diff['inDate'].iloc[i-1][8]+df_diff['inDate'].iloc[i-1][9])
            merge_dict[df_diff['gamer_id'].iloc[i-1]] = plus + 11 # 12월 + 1월(11일까지) = rct_diff
        elif int(df_diff['inDate'].iloc[i-1][5]+df_diff['inDate'].iloc[i-1][6]) == 1:
            merge_dict[df_diff['gamer_id'].iloc[i-1]] = 11 - int(df_diff['inDate'].iloc[i-1][8]+df_diff['inDate'].iloc[i-1][9]) # 11일(데이터 마지막 날짜와의 차이)  
        #merge_dict[df_diff['gamer_id'].iloc[i-1]] = df_diff['diff'].iloc[i-1]


# gamer_id unique 리스트 생성
gamerid_list = list(merge_dict.keys())

# 가장 최근 diff 리스트 생성
rct_diff_list = list(merge_dict.values())

# merge를 위한 데이터프레임 생성
df_rct_diff_merge = pd.DataFrame(data={'gamer_id' : gamerid_list,
                                       'rct_diff' : rct_diff_list})

In [ ]:
# merge
df = pd.merge(left=df, right=df_rct_diff_merge, on=['gamer_id'])

In [ ]:
df

gamer_id  ...  rct_diff
0         0001b282486938b2735e00ea3a65a6af52d648c15bfaa8...  ...         0
1         0001b282486938b2735e00ea3a65a6af52d648c15bfaa8...  ...         0
2         0001b282486938b2735e00ea3a65a6af52d648c15bfaa8...  ...         0
3         0001b282486938b2735e00ea3a65a6af52d648c15bfaa8...  ...         0
4         0001b282486938b2735e00ea3a65a6af52d648c15bfaa8...  ...         0
...                                                     ...  ...       ...
23017964  ff7e04acdb9d089c5a6fff1c1b4150156e6183dacc1f3b...  ...        10
23017965  ff7e04acdb9d089c5a6fff1c1b4150156e6183dacc1f3b...  ...        10
23017966  ff7e04acdb9d089c5a6fff1c1b4150156e6183dacc1f3b...  ...        10
23017967  ff7e04acdb9d089c5a6fff1c1b4150156e6183dacc1f3b...  ...        10
23017968  ff7e04acdb9d089c5a6fff1c1b4150156e6183dacc1f3b...  ...        10

[23017969 rows x 11 columns]

### risk ratio (위험 이탈비율) 계산

In [ ]:
df['risk_ratio'] = df['rct_diff'] / df['login_mean']

In [ ]:
df

gamer_id  ...  risk_ratio
0         0001b282486938b2735e00ea3a65a6af52d648c15bfaa8...  ...         0.0
1         0001b282486938b2735e00ea3a65a6af52d648c15bfaa8...  ...         0.0
2         0001b282486938b2735e00ea3a65a6af52d648c15bfaa8...  ...         0.0
3         0001b282486938b2735e00ea3a65a6af52d648c15bfaa8...  ...         0.0
4         0001b282486938b2735e00ea3a65a6af52d648c15bfaa8...  ...         0.0
...                                                     ...  ...         ...
23017964  ff7e04acdb9d089c5a6fff1c1b4150156e6183dacc1f3b...  ...    100000.0
23017965  ff7e04acdb9d089c5a6fff1c1b4150156e6183dacc1f3b...  ...    100000.0
23017966  ff7e04acdb9d089c5a6fff1c1b4150156e6183dacc1f3b...  ...    100000.0
23017967  ff7e04acdb9d089c5a6fff1c1b4150156e6183dacc1f3b...  ...    100000.0
23017968  ff7e04acdb9d089c5a6fff1c1b4150156e6183dacc1f3b...  ...    100000.0

[23017969 rows x 12 columns]

In [ ]:
print(set(df['risk_ratio']))
print(len(set(df['risk_ratio'])))

{0.0, 1.5, 2.0, 3.6666666666666665, 4.0, 5.0, 6.0, 1.4285714285714286, 0.375, 5.333333333333333, 10.0, 3.6, 1.125, 13.0, 0.75, 2.5, 9.0, 11.0, 12.0, 3.5, 3.0, 4.5, 5.5, 1.25, 1.0, 0.5, 0.25, 1.875, 7.0, 0.625, 1.75, 20000.0, 0.125, 8.0, 3.0000000000000004, 0.7272727272727273, 2.25, 40000.0, 0.3076923076923077, 1.2857142857142858, 0.8333333333333334, 60000.0, 1.4, 2.4, 0.9000000000000001, 0.4444444444444444, 0.3333333333333333, 0.27272727272727276, 0.2, 0.14285714285714285, 80000.0, 0.5714285714285714, 100000.0, 2.6666666666666665, 2.5714285714285716, 1.6666666666666667, 1.1666666666666667, 0.7142857142857143, 120000.0, 2.8, 0.30000000000000004, 0.8, 4.666666666666667, 1.8, 6.666666666666667, 140000.0, 0.8888888888888888, 0.8571428571428572, 0.6, 0.36363636363636365, 0.6666666666666666, 0.5454545454545455, 0.15384615384615385, 0.2222222222222222, 0.2857142857142857, 0.8571428571428571, 0.4, 0.8181818181818182, 10000.0, 1.1111111111111112, 1.1428571428571428, 30000.0, 1.2, 0.166666666666

In [ ]:
df[df['risk_ratio'] == 100000.0]

gamer_id  ...  risk_ratio
22814365  000221d5c17a8b3e3ba484359e982bd06ef2722d9d1ede...  ...    100000.0
22814366  000221d5c17a8b3e3ba484359e982bd06ef2722d9d1ede...  ...    100000.0
22814367  000221d5c17a8b3e3ba484359e982bd06ef2722d9d1ede...  ...    100000.0
22814368  000221d5c17a8b3e3ba484359e982bd06ef2722d9d1ede...  ...    100000.0
22814369  000221d5c17a8b3e3ba484359e982bd06ef2722d9d1ede...  ...    100000.0
...                                                     ...  ...         ...
23017964  ff7e04acdb9d089c5a6fff1c1b4150156e6183dacc1f3b...  ...    100000.0
23017965  ff7e04acdb9d089c5a6fff1c1b4150156e6183dacc1f3b...  ...    100000.0
23017966  ff7e04acdb9d089c5a6fff1c1b4150156e6183dacc1f3b...  ...    100000.0
23017967  ff7e04acdb9d089c5a6fff1c1b4150156e6183dacc1f3b...  ...    100000.0
23017968  ff7e04acdb9d089c5a6fff1c1b4150156e6183dacc1f3b...  ...    100000.0

[203604 rows x 12 columns]

### 평균접속주기 분포 확인

In [ ]:
df_rr_group = df[['gamer_id', 'login_mean']]
df_rr_group = df_rr_group.groupby(by=['gamer_id']).mean().reset_index()
df_rr_group['login_mean'] = df_rr_group['login_mean'].apply(lambda x: round(x, 4))
df_rr_group

gamer_id  login_mean
0       000010f658e9f47d01fd060a769e1d95395ac7c97c357d...      1.0000
1       000012d4f585a9bb5d6975e157b79b3538b04885ac19c6...      2.5000
2       00002a907f341c6113a96699f6da7d69bff34f29853c47...      0.0001
3       00003fb64d5509abbd5921dfd4736d9fc8d2c392a42c25...      0.0001
4       0000bbd7d65a8cc77af85023344fb46e3a00d3772dcbf3...      1.5000
...                                                   ...         ...
545907  fffeb0212f0f68f342421974ddf60b60b4468ca71f89fa...      1.0000
545908  ffff55ab88728761706a92ef75d68394c5dad8edd66818...      0.0001
545909  ffff84a159d96864ec6ee5ae6c9c4386d3060666bcc5a1...      0.0001
545910  ffffebdf9cff7c9d118dbb90fe145fbeb61c4978e1692d...      0.0001
545911  fffff1f88815e0ba807c3ff28ebcb94d9dbd8fce805eae...      0.0001

[545912 rows x 2 columns]

In [ ]:
df_rr_group['login_mean'].value_counts(normalize=True)

0.0001    0.585765
1.0000    0.218916
1.5000    0.056205
2.0000    0.049992
3.0000    0.020578
2.5000    0.011839
4.0000    0.011711
5.0000    0.006961
2.3333    0.005602
3.5000    0.005479
2.6667    0.004799
6.0000    0.004517
7.0000    0.003568
4.5000    0.002916
8.0000    0.002687
3.3333    0.002293
9.0000    0.002004
3.6667    0.001509
5.5000    0.000951
2.7500    0.000676
4.3333    0.000405
3.2500    0.000293
6.5000    0.000187
4.6667    0.000147
Name: login_mean, dtype: float64

# Labeling

### 1회 접속한 유저들의 이탈기준 정의

In [ ]:
df_onetime_group = df[['gamer_id', 'inDate', 'login_total', 'login_mean', 'rct_diff']].groupby(by=['gamer_id', 'inDate']).mean().reset_index()
df_onetime_group

gamer_id  ... rct_diff
0        000010f658e9f47d01fd060a769e1d95395ac7c97c357d...  ...        3
1        000010f658e9f47d01fd060a769e1d95395ac7c97c357d...  ...        3
2        000010f658e9f47d01fd060a769e1d95395ac7c97c357d...  ...        3
3        000010f658e9f47d01fd060a769e1d95395ac7c97c357d...  ...        3
4        000012d4f585a9bb5d6975e157b79b3538b04885ac19c6...  ...        5
...                                                    ...  ...      ...
1419952  fffeb0212f0f68f342421974ddf60b60b4468ca71f89fa...  ...        7
1419953  ffff55ab88728761706a92ef75d68394c5dad8edd66818...  ...       10
1419954  ffff84a159d96864ec6ee5ae6c9c4386d3060666bcc5a1...  ...       12
1419955  ffffebdf9cff7c9d118dbb90fe145fbeb61c4978e1692d...  ...       10
1419956  fffff1f88815e0ba807c3ff28ebcb94d9dbd8fce805eae...  ...        1

[1419957 rows x 5 columns]

In [ ]:
# 1회 접속한 유저들의 날짜별 첫접속 분포
onetime_df = df_onetime_group[df_onetime_group['login_total'] == 1]['inDate'].value_counts().sort_values().rename_axis('inDate').reset_index(name='count')
onetime_df = onetime_df.sort_values(by='inDate', ascending=True)

onetime_df['pct'] = onetime_df['count'] / onetime_df['count'].sum()
onetime_df['cum_count'] = onetime_df['count'].cumsum()
onetime_df['cum_pct'] = onetime_df['cum_count'] / onetime_df['count'].sum()

onetime_df.reset_index(drop=True)

inDate  count       pct  cum_count   cum_pct
0   2021-12-28  20610  0.064451      20610  0.064451
1   2021-12-29  27172  0.084972      47782  0.149423
2   2021-12-30  25603  0.080065      73385  0.229489
3   2021-12-31  21861  0.068363      95246  0.297852
4   2022-01-01  32588  0.101909     127834  0.399761
5   2022-01-02  24779  0.077489     152613  0.477250
6   2022-01-03  19427  0.060752     172040  0.538002
7   2022-01-04  19383  0.060614     191423  0.598616
8   2022-01-05  16369  0.051189     207792  0.649805
9   2022-01-06  19239  0.060164     227031  0.709969
10  2022-01-07  19172  0.059954     246203  0.769923
11  2022-01-08  21095  0.065968     267298  0.835891
12  2022-01-09  16704  0.052237     284002  0.888128
13  2022-01-10  15181  0.047474     299183  0.935602
14  2022-01-11  20593  0.064398     319776  1.000000

In [ ]:
# rr <= 1 유저들 중 rct_diff 최댓값 / 평균값
## (의미) rr <= 1 : 절대적으로 미이탈자로 구분되는 유저
df_rr_under_1 = df[['gamer_id', 'rct_diff', 'risk_ratio']].groupby(by='gamer_id').mean().reset_index()
df_rr_under_1 = df_rr_under_1[df_rr_under_1['risk_ratio'] <= 1]

In [ ]:
print(f"rr <= 1 유저의 최근접속경과일 최댓값: {df_rr_under_1['rct_diff'].max()}")
print(f"rr <= 1 유저의 최근접속경과일 평균값: {df_rr_under_1['rct_diff'].mean()}")
print(f"rr <= 1 유저의 최근접속경과일 중앙값: {statistics.median(list(set(df_rr_under_1['rct_diff'])))}")
print(f"rr <= 1 유저의 최근접속경과일 최솟값: {df_rr_under_1['rct_diff'].min()}")

rr <= 1 유저의 최근접속경과일 최댓값: 7
rr <= 1 유저의 최근접속경과일 평균값: 0.5730398226947557
rr <= 1 유저의 최근접속경과일 중앙값: 3.5
rr <= 1 유저의 최근접속경과일 최솟값: 0


In [ ]:
# "risk_ratio <= 1 유저들의 rct_diff(최근접속경과일) 평균값(4 == 3.5)을 기준으로 1회 접속한 유저들의 이탈 여부를 구분
# ==> 1회 접속한 유저의 71%(227,031명/319,776명)를 이탈자로 분류 
# ==> 최댓값이 아닌 중앙값으로 선정한 이유 : 최대한 많은 유저를 잠재이탈자로 생각하기 위함
# (?) 최댓값으로 해도 괜찮? 고민 필요

In [ ]:
# 1회 접속한 유저들 중 rct_diff <= 4 인 유저들(미이탈)의 risk_ratio 조정 ( login_mean = rct_diff != 0.0001 )
df.loc[(df['login_total'] == 1) & (df['rct_diff'] <= 4) & (df['rct_diff'] > 0), 'login_mean'] = df['rct_diff']

In [ ]:
# login_mean 조정 확인
df[df['gamer_id'] == '001cb885d1aa37df130c48d929925faf8e52051660ab2f6e82aeacff543c3b4d']

gamer_id  ...  risk_ratio
11789704  001cb885d1aa37df130c48d929925faf8e52051660ab2f...  ...     30000.0
11789705  001cb885d1aa37df130c48d929925faf8e52051660ab2f...  ...     30000.0
11789706  001cb885d1aa37df130c48d929925faf8e52051660ab2f...  ...     30000.0
11789707  001cb885d1aa37df130c48d929925faf8e52051660ab2f...  ...     30000.0
11789708  001cb885d1aa37df130c48d929925faf8e52051660ab2f...  ...     30000.0

[5 rows x 12 columns]

### 평균접속주기 1 / 1.5 유저 Advantage 적용


#### login_mean == 1

In [ ]:
# 평균접속주기 1 유저 : 1.98배 advantage 적용 ==> 47시간의 advantage 적용
df['login_mean'] = df['login_mean'].replace(1, 1.98)

#### login_mean == 1.5

In [ ]:
# 평균접속주기 1.5 유저 rct_diff 분포 확인
df_mean_group = df[['gamer_id', 'login_mean', 'rct_diff']]
df_mean_group = df_mean_group.groupby(by=['gamer_id']).mean().reset_index()
df_mean_group['login_mean'] = round(df_mean_group['login_mean'], 2)

print(f"평균접속주기 1.5 유저의 최근접속경과일 최댓값: {df_mean_group[df_mean_group['login_mean'] == 1.5]['rct_diff'].max()}")
print(f"평균접속주기 1.5 유저의 최근접속경과일 평균값: {df_mean_group[df_mean_group['login_mean'] == 1.5]['rct_diff'].mean()}")
print(f"평균접속주기 1.5 유저의 최근접속경과일 중앙값: {statistics.median(list(set(df_mean_group[df_mean_group['login_mean'] == 1.5]['rct_diff'])))}")
print(f"평균접속주기 1.5 유저의 최근접속경과일 최솟값: {df_mean_group[df_mean_group['login_mean'] == 1.5]['rct_diff'].min()}")

평균접속주기 1.5 유저의 최근접속경과일 최댓값: 11
평균접속주기 1.5 유저의 최근접속경과일 평균값: 2.9299286249714824
평균접속주기 1.5 유저의 최근접속경과일 중앙값: 5.5
평균접속주기 1.5 유저의 최근접속경과일 최솟값: 0


In [ ]:
# 평균접속주기 1.5 유저 rct_diff 별 점유율 확인
rct_1_5_df = df[df['login_mean'] == 1.5]['rct_diff'].value_counts().rename_axis('rct_diff').reset_index(name='count')

rct_1_5_df['pct'] = rct_1_5_df['count'] / rct_1_5_df['count'].sum()
rct_1_5_df['cum_count'] = rct_1_5_df['count'].cumsum()
rct_1_5_df['cum_pct'] = rct_1_5_df['cum_count'] / rct_1_5_df['count'].sum()

rct_1_5_df.reset_index(drop=True)

rct_diff    count       pct  cum_count   cum_pct
0          0  1864679  0.587772    1864679  0.587772
1          1   378122  0.119189    2242801  0.706961
2          2   198231  0.062485    2441032  0.769446
3          3   172465  0.054363    2613497  0.823810
4          4   116685  0.036781    2730182  0.860590
5          5   104971  0.033088    2835153  0.893679
6          6    86191  0.027169    2921344  0.920847
7          7    79749  0.025138    3001093  0.945985
8          8    62039  0.019556    3063132  0.965541
9          9    56156  0.017701    3119288  0.983242
10        10    38160  0.012029    3157448  0.995271
11        11    15004  0.004729    3172452  1.000000

In [ ]:
# 1.5 평균값 기준 rct_diff <= 3 인 유저들한테 1.98 어드밴티지 적용
df.loc[(df['login_mean'] == 1.5) & (df['rct_diff'] <= 3), 'login_mean'] = df['login_mean'] * 1.98

In [ ]:
# login_mean 조정 확인
df[df['gamer_id'] == '0307bd62ce0df8a22f72373113d3a8d468b0ec09ffb367e35bc9231b7b96ffb3']

gamer_id  ...  risk_ratio
90687  0307bd62ce0df8a22f72373113d3a8d468b0ec09ffb367...  ...         2.0
90688  0307bd62ce0df8a22f72373113d3a8d468b0ec09ffb367...  ...         2.0
90689  0307bd62ce0df8a22f72373113d3a8d468b0ec09ffb367...  ...         2.0
90690  0307bd62ce0df8a22f72373113d3a8d468b0ec09ffb367...  ...         2.0
90691  0307bd62ce0df8a22f72373113d3a8d468b0ec09ffb367...  ...         2.0
...                                                  ...  ...         ...
90799  0307bd62ce0df8a22f72373113d3a8d468b0ec09ffb367...  ...         2.0
90800  0307bd62ce0df8a22f72373113d3a8d468b0ec09ffb367...  ...         2.0
90801  0307bd62ce0df8a22f72373113d3a8d468b0ec09ffb367...  ...         2.0
90802  0307bd62ce0df8a22f72373113d3a8d468b0ec09ffb367...  ...         2.0
90803  0307bd62ce0df8a22f72373113d3a8d468b0ec09ffb367...  ...         2.0

[117 rows x 12 columns]

### Churn_proba

In [ ]:
# risk_ratio 업데이트
df['risk_ratio_2'] = df['rct_diff'] / df['login_mean']

In [ ]:
# 시그모이드
def sigmoid(x):
    sig = 1 / (1 + math.exp(-x))
    return sig

In [ ]:
# 이탈확률 계산
df['churn_proba'] = df['risk_ratio_2'].apply(lambda x: sigmoid(x))

In [ ]:
# 유저별 이탈확률 데이터프레임 생성
df_churn_proba = df[['gamer_id', 'churn_proba', 'risk_ratio_2']]
df_churn_proba = df_churn_proba.groupby(by=['gamer_id']).mean().reset_index()

In [ ]:
# 이탈확률별 개수 확인을 위한 데이터테이블 생성
df_churn_proba_group = df_churn_proba.groupby(by=['churn_proba', 'risk_ratio_2']).count().reset_index()
df_churn_proba_group = df_churn_proba_group.rename(columns={'gamer_id':'count'})

In [ ]:
# 이탈확률별 count, percent, cumulative count, cumulative percent 열 생성
df_churn_proba_group['pct'] = df_churn_proba_group['count'] / df_churn_proba_group['count'].sum()
df_churn_proba_group['cum_count'] = df_churn_proba_group['count'].cumsum()
df_churn_proba_group['cum_pct'] = df_churn_proba_group['cum_count'] / df_churn_proba_group['count'].sum()
df_churn_proba_group = df_churn_proba_group.sort_values(by='churn_proba', ascending=True)

In [ ]:
df_churn_proba_group

churn_proba   risk_ratio_2  count       pct  cum_count   cum_pct
0        0.500000       0.000000  87953  0.161112      87953  0.161112
1        0.527749       0.111111     10  0.000018      87963  0.161130
2        0.527749       0.111111     16  0.000029      87979  0.161160
3        0.527749       0.111111     52  0.000095      88031  0.161255
4        0.527749       0.111111     25  0.000046      88056  0.161301
...           ...            ...    ...       ...        ...       ...
4643     1.000000   70000.000000  19383  0.035506     373872  0.684858
4642     1.000000   60000.000000  16369  0.029985     354489  0.649352
4641     1.000000   50000.000000  19239  0.035242     338120  0.619367
4644     1.000000   80000.000000  19427  0.035586     393299  0.720444
4650     1.000000  140000.000000  20610  0.037753     545912  1.000000

[4651 rows x 6 columns]

### Labeling

평균주기 2일인 유저의 평균 risk_ratio : 1.69 <br>
평균주기 3일인 유저의 평균 risk_ratio : 1.16 <br>
평균주기 4일인 유저의 평균 risk_ratio : 1.01 <br>
평균주기가 짧은 유저들에 대해 더 집중할 필요있음 <br>
평균주기 2일인 유저의 평균 risk_ratio를 기준으로 평균인 1.69이 labeling의 기준 risk_ratio 최소기준이 될 것 <br>
<br>
risk_ratio == 2 를 이탈 기준으로 정의

In [ ]:
df_y = df[['gamer_id', 'risk_ratio_2']]
df_y = df_y.groupby(by='gamer_id').mean().reset_index()
df_y['y'] = ""
df_y

gamer_id   risk_ratio_2 y 
0       000010f658e9f47d01fd060a769e1d95395ac7c97c357d...       1.515152   
1       000012d4f585a9bb5d6975e157b79b3538b04885ac19c6...       2.000000   
2       00002a907f341c6113a96699f6da7d69bff34f29853c47...   80000.000000   
3       00003fb64d5509abbd5921dfd4736d9fc8d2c392a42c25...  130000.000000   
4       0000bbd7d65a8cc77af85023344fb46e3a00d3772dcbf3...       0.673401   
...                                                   ...            ... ..
545907  fffeb0212f0f68f342421974ddf60b60b4468ca71f89fa...       3.535354   
545908  ffff55ab88728761706a92ef75d68394c5dad8edd66818...  100000.000000   
545909  ffff84a159d96864ec6ee5ae6c9c4386d3060666bcc5a1...  120000.000000   
545910  ffffebdf9cff7c9d118dbb90fe145fbeb61c4978e1692d...  100000.000000   
545911  fffff1f88815e0ba807c3ff28ebcb94d9dbd8fce805eae...       0.505051   

[545912 rows x 3 columns]

In [ ]:
# Labeling
df_y.loc[(df_y['risk_ratio_2'] >= 2), 'y'] = 1
df_y.loc[(df_y['risk_ratio_2'] < 2), 'y'] = 0
df_y = df_y.rename(columns={'risk_ratio_2':'risk_ratio_final'})
df_y = df_y[['gamer_id', 'risk_ratio_final', 'y']]

In [ ]:
df = pd.merge(left=df, right=df_y, on='gamer_id')

In [ ]:
df

gamer_id  ...  y
0         0001b282486938b2735e00ea3a65a6af52d648c15bfaa8...  ...  0
1         0001b282486938b2735e00ea3a65a6af52d648c15bfaa8...  ...  0
2         0001b282486938b2735e00ea3a65a6af52d648c15bfaa8...  ...  0
3         0001b282486938b2735e00ea3a65a6af52d648c15bfaa8...  ...  0
4         0001b282486938b2735e00ea3a65a6af52d648c15bfaa8...  ...  0
...                                                     ...  ... ..
23017964  ff7e04acdb9d089c5a6fff1c1b4150156e6183dacc1f3b...  ...  1
23017965  ff7e04acdb9d089c5a6fff1c1b4150156e6183dacc1f3b...  ...  1
23017966  ff7e04acdb9d089c5a6fff1c1b4150156e6183dacc1f3b...  ...  1
23017967  ff7e04acdb9d089c5a6fff1c1b4150156e6183dacc1f3b...  ...  1
23017968  ff7e04acdb9d089c5a6fff1c1b4150156e6183dacc1f3b...  ...  1

[23017969 rows x 16 columns]

# 추출

In [ ]:
df.to_csv('df_labeled.csv', mode='w')